In [ ]:
from io import BytesIO
import numpy as np
import requests
import xarray as xr
import hvplot.xarray
import hvplot.pandas

def download_data(url, **kwargs):
    r = requests.get(url)
    buf = BytesIO(r.content)
    return xr.open_dataset(buf, **kwargs)    

# Generate data remotely
cmda_url = 'http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/svc/scatterPlot2Vars'
model1, var1 = 'NCAR_CAM5', 'tos'
model2, var2 = 'NCAR_CAM5', 'wap'
nSample = 10000
width, height = 1000, 500
v1 = f'{model1}:{var1}'
v2 = f'{model2}:{var2}'
query = dict(model1=model1, var1=var1, pres1=-999999, model2=model2, var2=var2, 
             pres2=900, lonS=0, lonE=360, latS=-20, latE=20, timeS=200501, timeE=200512, nSample=nSample)
r = requests.get(cmda_url, params=query)
data_url = r.json()['dataUrl']

# Download data into xarray Dataset object
ds = download_data(data_url, decode_times=False).rename(data1=v1, data2=v2)
ds

In [ ]:
pts = ds.stack(pt=['time', 'lat', 'lon']).reset_index('pt').dropna('pt')
pts = pts.isel(pt=np.random.choice(np.arange(pts.pt.size), nSample, replace=False))
corr = np.corrcoef(pts[v1], pts[v2])[0, 1]
pts.reset_coords().to_dataframe().hvplot.scatter(x=v1, y=v2, title=f'Correlation: {corr:1.2}', width=width, height=height)

In [ ]:
pts.hvplot.hist(y=v1, width=width, height=height, normed=True)

In [ ]:
pts.hvplot.hist(y=v2, width=width, height=height, bin_range=(-.1, .06), normed=True)